In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code
from sample.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter1d
from scipy import stats
import statsmodels.api as sm
from sklearn.neighbors import KernelDensity
window=800
infos = pickle.load(open(f'{path_code}/MscThesis/Results/straight_bait_{window}.pick', "rb"))
def get_angle(xa,ya,xb,yb):
    dot_product = (xa*xb+ya*yb)/np.sqrt((xa**2+ya**2)*(xb**2+yb**2))
    angle = (np.arccos(dot_product) / (2 * np.pi) * 360)*(1-2*((ya * xb - yb * xa) >= 0))
    return(angle)
infos['abs_angle_to_P']=np.abs(infos['angle_to_P'])
infos['abs_curvature']=np.abs(infos['curvature'])
infos['curvature_scaled']=infos['curvature']*np.sqrt(infos['growth'])
infos['abs_curvature_sq']=np.abs(infos['curvature'])*np.sqrt(infos['growth'])
infos['signed_straight']=(1-infos['straightness'])*(1-2*(infos['angle']<0))
infos['inv_dens']=1/infos['density']
infos['side_cross']=infos.apply (lambda row: comments[row['plate']] if row['plate'] in comments.keys() else 'None', axis=1)
infos['x']=infos['x'].astype(float)
infos['vx']=infos['vx'].astype(float)
infos['y']=infos['y'].astype(float)
infos['vy']=infos['vy'].astype(float)
infos['xinit']=infos['xinit'].astype(float)
infos['yinit']=infos['yinit'].astype(float)
blur = 20
infos['v'] = np.sqrt((infos['vx']**2+infos['vy']**2).astype(float))
infos['gd'] = np.sqrt((infos[f'grad_density_x{blur}']**2+infos[f'grad_density_y{blur}']**2).astype(float))
infos['spvgd']=(infos['vx']*infos[f'grad_density_x{blur}']+infos['vy']*infos[f'grad_density_y{blur}'])/(infos['v']*infos['gd'])
infos['angle_vgd']=get_angle(-infos[f'grad_density_x{blur}'],-infos[f'grad_density_y{blur}'],infos['vx'],infos['vy'])
infos['angle_vgd2']=get_angle(-infos[f'grad_density_x{blur}'],-infos[f'grad_density_y{blur}'],infos['xinit'],infos['yinit'])
infos['angle_Pgd']=infos['angle_vgd']+infos['angle_to_P']
infos['angle_Ngd']=infos['angle_vgd']+infos['angle_to_N']
# infos['residual']=infos['speed']-f(infos['spvgd'])
corrected = infos.loc[(infos["straightness"] <= 1)& (infos["speed"] >=25)& (infos["speed"] <400)&(infos["straightness"] > 0.95)&(infos["density"]>0)]
baits = corrected.loc[corrected['treatment']=='baits']
no_baits = corrected .loc[corrected ['treatment']=='25']
no_baits_new = corrected .loc[corrected ['treatment']=='25*']

In [2]:
def getKernelDensityEstimation(values, x, bandwidth = 0.2, kernel = 'gaussian'):
    model = KernelDensity(kernel = kernel, bandwidth=bandwidth)
    model.fit(values[:, np.newaxis])
    log_density = model.score_samples(x[:, np.newaxis])
    return np.exp(log_density)
def bestBandwidth(data, minBandwidth = 0.1, maxBandwidth = 2, nb_bandwidths = 30, cv = 30):
    """
    Run a cross validation grid search to identify the optimal bandwidth for the kernel density
    estimation.
    """
    from sklearn.model_selection import GridSearchCV
    model = GridSearchCV(KernelDensity(),
                        {'bandwidth': np.linspace(minBandwidth, maxBandwidth, nb_bandwidths)}, cv=cv) 
    model.fit(data[:, None])
    return model.best_params_['bandwidth']

In [3]:
from sklearn.model_selection import GridSearchCV

In [ ]:
plt.close('all')
plate_select = corrected.loc[corrected ['plate']==94]
# plate_select = corrected.loc[corrected ['plate'].isin(right_crossing)]
# straight_going = corrected.loc[corrected['plate'].isin(non_turning)]
abcisse = 'density'
ordinate = 'speed'
tab = plate_select
baits_sort = tab.sort_values(abcisse)
N=400
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('density($mm.mm^{-2}$)')

ax.set_ylabel('speed ($\mu m .h^{-1}$)')
ax.set_xlim(0,6)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab['plate'].values):
    select = tab.loc[tab['plate']==plate]
    ax.scatter(select[abcisse],select[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
# ax.legend()

In [4]:
plt.close('all')
plate_select = corrected.loc[corrected ['plate']==94]
# plate_select = corrected.loc[corrected ['plate'].isin(right_crossing)]
# straight_going = corrected.loc[corrected['plate'].isin(non_turning)]
abcisse = 'spvgd'
ordinate = 'speed'
tab = plate_select
baits_sort = tab.sort_values(abcisse)
N=400
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_ylabel('speed ($\mu m .h^{-1}$)')

ax.set_xlabel('normalized scalar product speed.density gradient')
ax.set_xlim(-1,1)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab['plate'].values):
    select = tab.loc[tab['plate']==plate]
    ax.scatter(select[abcisse],select[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
# ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [19]:
from statsmodels.nonparametric.bandwidths import bw_silverman, bw_scott, select_bandwidth
dummy_baits = [298,319,373,344]
P_N_crossing_P = [420,433,405,425,384]
P_N_crossing_N = [423,436,409,400,403]
plate_select = corrected.loc[corrected ['plate'].isin([344])]
# plate_select = corrected.loc[corrected ['treatment']=='25*']

plt.close('all')
fig = plt.figure()
bins = np.linspace(0, 400, 50)
ax = fig.add_subplot(111)
ax.hist(corrected.loc[corrected['plate'].isin(dummy_baits)]['speed'],bins,alpha=0.25,label='Dummy baits (homogeneous soluble)',density=True)
ax.hist(corrected.loc[corrected['plate'].isin(P_N_crossing_P)]['speed'],bins,alpha=0.25,label='P&N baits crossing P (heterogeneous rock form)',density=True)
ax.hist(corrected.loc[corrected['plate'].isin(P_N_crossing_N)]['speed'],bins,alpha=0.25,label='P&N baits crossing N (heterogeneous rock form)',density=True)
ax.hist(corrected.loc[corrected ['treatment']=='25*']['speed'],bins,alpha=0.25,label='No baits (homogeneous soluble)',density=True)
# ax.hist(corrected.loc[corrected['plate'] == 69]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# ax.hist(corrected.loc[corrected['plate'] == 102]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# x = np.linspace(-6,6,100)
x = np.linspace(0,400,100)

# bandwidth = 5
# data = corrected.loc[corrected['plate'] == 94]['speed']
# data = plate_select['speed']

# cv_bandwidth = bestBandwidth(data,minBandwidth=2,maxBandwidth=20)
# # cv_bandwidth = 4


# kde = getKernelDensityEstimation(data, x, bandwidth=cv_bandwidth)
# ax.plot(x, kde, alpha = 0.8, label = f'bandwidth = {round(cv_bandwidth, 2)}')
# # ax.plot(x, stats_models_cv, alpha = 0.8, label = f'bandwidth = {round(cv_bandwidth, 2)}')
# ax.hist(data,30,density=True)
ax.set_xlabel(r'speed ($\mu m.h^{-1} $)')
plt.legend(loc='upper right')
ax.set_ylabel(r'density')
plt.savefig('plot.eps', format='eps')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
from statsmodels.nonparametric.kernel_density import KDEMultivariate
stats_models_cv = KDEMultivariate(data, 'c', bw = 'cv_ml').pdf(x)

In [15]:
fig = plt.figure()
bins = np.linspace(0, 400, 50)
ax = fig.add_subplot(111)
from unidip import UniDip
import unidip.dip as dip
data = np.msort(data)
print(dip.diptst(data))
intervals = UniDip(data,ntrials=2000,debug=True,mrg_dst = 0.1).run()
print(intervals)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.01110841275008323, 0.17182817182817184, (425, 984))
[(0, 1739)]
